# Matrix Factorization

The problem we propose to address here is that of __rating prediction__. The data we have is a rating history: ratings of users for items in the interval [1,5] (1 to 5 stars rating). We can put all this data into a sparse matrix called $R$.

The matrix $R$ is sparse (more than 99% of the entries are missing), and our goal is to __predict__ the missing entries. One of the solution for this rating prediction is _matrix factorization_. This is fundamentally linked to SVD, which stands for Singular Value Decomposition.

Here is the matrix factorization:

$R = M\sum U^T$

To be clear, SVD is an algorithm that takes the matrix $R$ as an input, and it gives you $M$, $\sum$ and $U$.

## References:

1. http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/
2. http://www.albertauyeung.com/post/python-matrix-factorization/
3. https://arxiv.org/pdf/1503.07475.pdf

In [8]:
import numpy as np

In [3]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P, Q)
        e = 0

        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)

                    for k in range(K):
                        e = e + (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break
    return P, Q.T
                                                                                  

In [6]:
R = [[5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4]]

R = np.array(R)

N = len(R)
M = len(R[0])
K = 2

# P = np.random.rand(N, K)
# Q = np.random.rand(M, K)

P = np.array([[ 0.76429108,  0.68098794],
 [ 0.33121903,  0.56332544],
 [ 0.00385484,  0.52690095],
 [ 0.40541877,  0.49737146],
 [ 0.17445866,  0.52815968]])
Q = np.array([[ 0.00909807,  0.57423495],
 [ 0.57250499,  0.06490164],
 [ 0.39694569,  0.77008759],
 [ 0.95925171,  0.1277469 ]])

nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)
print(nR)

[[ 5.04768771  2.78483412  5.50230144  0.99807947]
 [ 3.92570891  2.16834716  4.40570898  0.99751618]
 [ 1.11464726  0.66887894  3.92666805  4.96624064]
 [ 0.94499662  0.5644064   3.19483872  3.97552371]
 [ 2.60432914  1.47677322  4.84797291  4.03121858]]


In [2]:
class MatrixFactorization:
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimension
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R 
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i + 1) % 10 == 0:
                print('Iterations: {}; error = {}'.format(i + 1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic gradient descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update bias
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_u[j])
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
        
    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j,:].T)
        return prediction
    
    def full_matrix(self):
        """
        Compute the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:, np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [5]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

mf = MatrixFactorization(R, K=2, alpha=0.1, beta=0.01, iterations=50)
res = mf.train()
print(mf.full_matrix())

Iterations: 10; error = 0.5795806634257686
Iterations: 20; error = 0.177101635232628
Iterations: 30; error = 0.06552744650804607
Iterations: 40; error = 0.04021978140460307
Iterations: 50; error = 0.03464785215202055
[[ 4.98860367  2.9978165   3.42637656  1.01050362]
 [ 3.99547931  2.71239392  3.55887549  1.01087669]
 [ 1.01188101  0.99702403  5.72188769  4.98428064]
 [ 1.00909374  0.34626056  4.64210952  3.99514875]
 [ 1.44227809  1.01402861  4.98933919  3.9982778 ]]
